# Walking friction surface creation

Clean up various rasters and combining them into a walking friction surface. This forms half of the multi-modal friction surface and can also be used as a standalone analysis tool (for walking-only analysis). This will later be merged with an on-road friction surface for a final product.

In [74]:
import os, sys

from datetime import datetime

import common_rasterio_ops as rast_ops # custom functions

import numpy as np
from numpy import pi, log, tan, empty, float32, arctan, rad2deg, gradient
from numpy import arctan2, reshape, where
from scipy.ndimage import gaussian_gradient_magnitude

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, Polygon

In [2]:
data_dir = r'P:\PAK\\Code\Accessibility\Inputs'
in_dem = r'P:\PAK\\GEO\Elevation'
in_lc = r'P:\PAK\\GEO\Landcover\ICIMOD'
out_dir = r'P:\PAK\\Code\Accessibility\Intermediate_Rasters'
fric_dir = r'P:\PAK\\Code\Accessibility\Friction_Surface'

Projections

In [3]:
dest_crs = 'EPSG:32642' # this is a Pakistani UTM projection, assign correct projection for project area

Today's date

In [4]:
today = datetime.today().strftime('%y%m%d')

Load Shapefile of area of interest (in our case Khyber Pakhtunkhwa province, Pakistan) to clip the final data

In [5]:
aoi_pth = r'P:\PAK\GEO\Boundaries\OCHA\pak_admbnda_adm1_ocha_pco_gaul_20181218.shp'

In [6]:
aoi = gpd.read_file(aoi_pth)

In [7]:
aoi = aoi[aoi['ADM1_EN'] == 'Khyber Pakhtunkhwa']
aoi = aoi.to_crs(dest_crs)

In [8]:
# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(20000)

In [9]:
aoi.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
2,27.479368,9.901186,Khyber Pakhtunkhwa,PK2,None,None,None,Pakistan,PK,2018-11-30,2018-12-18,None,"POLYGON ((505910.537 3561734.747, 505840.700 3..."


# Processing

Load in and process various input rasters for off-road travel. Note that the decision to use the landcover as the reference layer (in terms of projection, cell size, etc.) is arbitrary and the DEM could easily be used for such instead.

## Reclassify landcover

In [10]:
# Build a "lookup array" where the index is the original value and the value
# is the reclassified value.  Setting all of the reclassified values is cheap 
# because the memory is only allocated once for the lookup array.

lookup = np.arange(256, dtype=np.float32)

# Replacement values are the divisors of walking speeds specific to that landcover type -- so, a value of 2 means "divide the walking speed by 2"
# THESE ARE EXAMPLE VALUES AND MUST BE REPLACED FOR YOUR CONTEXT -- refer to "Spatial Analysis by Cost Functions" by Irmela Herzog (2020) for guidance

lookup[1] = 2
lookup[2] = 2
lookup[3] = 2
lookup[4] = 2
lookup[5] = 2
lookup[6] = 2
lookup[7] = 1.2
lookup[8] = 1.5
lookup[9] = 1.8
lookup[10] = 1.5
lookup[11] = 1.2
lookup[12] = 1.19
lookup[13] = 1.6
lookup[14] = 5
lookup[255] = 1.5


In [11]:
lc_pth = r'pakistan_icimod_landcover_2010_32642.tif'

In [12]:
with rasterio.open(os.path.join(in_lc,lc_pth)) as lc_src:
    # Read as numpy array
    lc_array = lc_src.read()
    lc_profile = lc_src.profile
    lc_transform = lc_src.transform
    
    # Reclassify in a single operation using broadcasting
    lc_array = lookup[lc_array].astype(np.float32)


In [13]:
lc_array.shape

(1, 21580, 29320)

In [14]:
lc_array.dtype

dtype('float32')

In [15]:
lc_transform

Affine(28.23254382673943, 0.0, 489325.4971,
       0.0, -31.766168813716423, 4128951.9286)

In [16]:
res = str(int(np.abs(lc_transform.e))) + 'm'

In [17]:
res

'31m'

## Rivers and bridges as obstacles

Download rivers, transform to geodataframe in correct projection

In [18]:
riv_path = r'osm_river_utm.shp'

In [19]:
# local file import
rivs = gpd.read_file(os.path.join(data_dir,riv_path),driver="ESRI Shapefile")

# minor cleanup
rivs = rivs.reset_index()
rivs_slim = rivs[['geometry']]
rivs_slim['exist'] = 0
rivs_slim = rivs_slim.to_crs(dest_crs)

C:\Users\wb467985\Anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [20]:
# create a generator containing geometry, value pairs for rivers

riv_shapes = ((geom,exist) for geom, exist in zip(rivs_slim.geometry,rivs_slim['exist']))

# This will give the raster the size and dimensions of the landcover raster -- areas not covered by rivers will be 1.

riv_rast = features.rasterize(riv_shapes,\
                  out_shape = (lc_profile['height'],\
                               lc_profile['width']),\
                  transform=lc_profile['transform'],
                  all_touched=True,
                  fill=1,
                  dtype = np.float32)


**Optional step, change the `using_bridges` parameter to trigger / disable**:

In some areas of the Himalayas foot bridges are an important means of crossing rivers, which tend to be swift moving and impassible. These bridges can save hours in extreme cases. If this is true for your area of interest, use the code below to load in bridges, snap them to river lines, and add them as crossable cells on your river raster.</br>This step is set to 'no' by default as it's a niche use case. 

In [21]:
# change this to trigger / disable sequence below 
using_bridges = 'no'

In [22]:
# Download bridges, transform to geodataframe in correct projection, snap to nearest river
if using_bridges == 'yes':

    # swap in file path of your bridges layer
    brdgs = gpd.read_file(os.path.join(data_dir,vect_dir,"bridges_osm_210710.shp"),driver="ESRI Shapefile")
    brdgs = brdgs.to_crs(dest_crs)

    # snapping to rivers
    # from: https://gis.stackexchange.com/questions/306838/snap-points-shapefile-to-line-shapefile-using-shapely</br>

    brdgs_snapped = [shapely.ops.snap(i,j, 2000) for i, j in zip(brdgs['geometry'],rivs_slim['geometry']) ]
    
    brdgs_snapped = gpd.GeoDataFrame(brdgs_snapped)
    brdgs_snapped.rename({0:'geometry'},axis=1,inplace=True)
    brdgs_snapped = brdgs_snapped.set_geometry('geometry').set_crs(dest_crs)
    brdgs_snapped['exist'] = 1

    # if you want to save these out
    # brdgs_snapped.to_file(os.path.join(data_dir,vect_dir,"bridges_osm_210710.shp"),driver="ESRI Shapefile")

    # generator of vector shapes and values (existean)

    brdg_shapes = ((geom,exist) for geom, exist in zip(brdgs_snapped.geometry,brdgs_snapped['exist']))

    # Create a bridge raster layer we can overlay on the rivers -- areas not covered by bridges will be 0

    brdg_rast = features.rasterize(brdg_shapes,\
                      out_shape = (lc_profile['height'],\
                                   lc_profile['width']),\
                      transform=lc_profile['transform'],
                      all_touched=True,
                      fill=0,
                      dtype = np.float32)

    # Add rasters together and export as final river raster layer
    
    riv_rast_fin = riv_rast + brdg_rast # add rasters together so bridges punch "holes" in rivers

else:
    # If not using bridges
    riv_rast_fin = riv_rast

In [23]:
riv_rast_fin.shape

(21580, 29320)

In [24]:
riv_rast_fin.dtype

dtype('float32')

## Roads to walking surface mask raster

We assume that people walking on roads and paths are not affected by landcover. To model this we turn roads into a raster with value = 1 (for 1 * speed). Then we merge it with the landcover raster for a final walking speed modifier raster

In [25]:
# your roads file path here
roads_path = r'master_transport_Sep23.gpkg'

In [26]:
rds = gpd.read_file(os.path.join(data_dir,roads_path),driver="GPKG")

In [27]:
# assign 1 value to represent existence of road
rds['exist'] = 1

# generator of vector shapes and values (boolean)
rds_shapes = ((geom,exist_val) for geom, exist_val in zip(rds.geometry,rds['exist']))

# This will give the raster the size and dimensions of the landcover raster -- areas not covered by roads will be 0.

rd_mask_rast = features.rasterize(rds_shapes,\
                  out_shape = (lc_profile['height'],\
                               lc_profile['width']),\
                  transform=lc_profile['transform'],
                  all_touched=True,
                  fill=0,
                  dtype = np.uint8)


In [28]:
print('shape: ' + str(rd_mask_rast.shape) + '\ndtype: ' + str(rd_mask_rast.dtype))

shape: (21580, 29320)
dtype: uint8


First combine the rivers with the landcover raster, inserting a `600000` divider where rivers exist, so crossing rivers without a bridge has a huge cost. Then combine with the road mask, inserting a `1` multiplier where roads are. The order is important, so roads overwrite rivers (implicitly via bridges, which are not reliably recorded in many roads datasets)
</br></br>Note that if landcover *multipliers* instead of *dividers* are used, you need to invert this and use a very small decimal value for the rivers.

In [29]:
# make rivers impassible
walkspeed_mod_rast = np.where(riv_rast_fin == 0, 600000, lc_array)
# treat roads as bridging rivers by default
walkspeed_mod_rast = np.where(rd_mask_rast == 1, 1, walkspeed_mod_rast)

In [30]:
print('shape: ' + str(walkspeed_mod_rast.shape) + '\ndtype: ' + str(walkspeed_mod_rast.dtype))

shape: (1, 21580, 29320)
dtype: float64


In [31]:
# check that the minimum value is correct
np.min(walkspeed_mod_rast)

1.0

## Base walking speeds from DEM

#### DEM to slope

First import the DEM and transform it to the same CRS, cell resolution, and dimensions as the landcover layer. This enables raster math between the layers and any other arrays derived from them.

In [32]:
with rasterio.open(os.path.join(in_dem,'DEM_KPK_UTM.tif')) as dem_src:
    # Read as numpy array
    dem_array = dem_src.read(1)
    dem_transform = dem_src.transform
    dem_profile = dem_src.profile


In [33]:
# must reproject the DEM to the same projection and cell-size as the landcover raster

# create a blank array of the correct dimensions to populate while reprojecting
dem_array_reproj = np.zeros(lc_array.shape, np.float32)

# reproject and warp to correct shape
with rasterio.Env():
    reproject(
        dem_array,
        dem_array_reproj,
        src_transform=dem_transform,
        src_crs=dem_profile['crs'],
        dst_transform=lc_profile['transform'],
        dst_crs=lc_profile['crs'],
        resampling=Resampling.cubic)

In [34]:
dem_array_reproj.shape


(1, 21580, 29320)

In [38]:
# use get_slope function from common_rasterio_ops
slope = rast_ops.get_slope(dem_array_reproj,mode='fraction')

In [39]:
# remove artefacts that will produce slopes > 100%
slope = np.where(slope>1,1,slope)

In [40]:
print('shape: ' + str(slope.shape) + '\ndtype: ' + str(slope.dtype))

shape: (1, 21580, 29320)
dtype: float32


Calculate walking speeds over the slope using Irmischer and Clarke's 2018 walking speed formula.

In [41]:
# Irmischer and Clarke have a generic off-road speed formula but we don't use this given that we adjust by specific landcover type.  Rather, we modify their on-road speed.
# We include the I+C off-road formula below for reference

# walkspeed_offroad = (0.11 + (0.67 * np.exp(-np.square((slope*100) + 2) / 3600))) * 3.6 # I-C off-road
walkspeed_onroad = (0.11 + np.exp(-np.square((slope*100) + 5) / 3600)) * 3.6

In [42]:
walkspeed_onroad

array([[[3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
         3.9710863],
        [3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
         3.9710863],
        [3.9710863, 3.9710863, 3.9710863, ..., 3.9710863, 3.9710863,
         3.9710863],
        ...,
        [3.947854 , 3.9466481, 3.9454958, ..., 3.9710863, 3.9710863,
         3.9710863],
        [3.9502766, 3.9490733, 3.9479506, ..., 3.9710863, 3.9710863,
         3.9710863],
        [3.9525816, 3.9513788, 3.9502866, ..., 3.9710863, 3.9710863,
         3.9710863]]], dtype=float32)

In [43]:
walkspeed_base = walkspeed_onroad
# walkspeed_base = np.where(rd_mask_rast == 1,walkspeed_onroad,walkspeed_offroad) # included for reference purposes, in situations where you don't want to adjust by landcover

In [48]:
print('min: ' + str(np.min(walkspeed_base)) + '\nmax: ' + str(np.min(walkspeed_base)) + '\nshape: ' + str(walkspeed_base.shape))

min: 0.5643742
max: 0.5643742
shape: (1, 21580, 29320)


#### Vertical distances

Calculate the additional vertical distance covered when crossing a cell (the rise, in addition to the run represented by the cell's resolution).

In [57]:
slope

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.0196207 , 0.02049321, 0.02131746, ..., 0.        ,
         0.        , 0.        ],
        [0.01783525, 0.01872793, 0.01955051, ..., 0.        ,
         0.        , 0.        ],
        [0.0160927 , 0.01700722, 0.01782777, ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)

In [59]:
vert_dist_simple = 1 / np.cos(slope)

Calculate the additional distance associated with zig-zagging paths - the zig goes sideways halfway up the cell, the zag sideways up the other half. We do not consider circumstances with more than 2 zig zags per cell -- possibly problematic if using large cells (1km+)

In [60]:
# The switchback cutoff value is somewhat arbitrary and perhaps even varies by cultue / context. 
# We use one of the higher values found in the literature as residents of the Himalayas might be expected to have a high tolerance for walking up steep hills

switchback_cutoff = 0.30

In [61]:
vert_dist_switchback = np.tan(slope) / np.sin(switchback_cutoff)

Combine the two arrays into one walking cost array, forcing walkers to use zig-zagging switchbacks while crossing terrain above a cutoff slope of `30%` (0.30). 

In [62]:
vert_dist_switchback = np.where(slope <= switchback_cutoff,vert_dist_simple,vert_dist_switchback)

In [63]:
vert_dist_switchback

array([[[1.       , 1.       , 1.       , ..., 1.       , 1.       ,
         1.       ],
        [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
         1.       ],
        [1.       , 1.       , 1.       , ..., 1.       , 1.       ,
         1.       ],
        ...,
        [1.0001925, 1.00021  , 1.0002272, ..., 1.       , 1.       ,
         1.       ],
        [1.000159 , 1.0001754, 1.0001911, ..., 1.       , 1.       ,
         1.       ],
        [1.0001295, 1.0001446, 1.0001589, ..., 1.       , 1.       ,
         1.       ]]], dtype=float32)

In [65]:
# make float32 to reduce file sizes on export
vert_dist_simple = vert_dist_simple.astype(np.float32)
vert_dist_switchback = vert_dist_switchback.astype(np.float32)

Mask and export the vertical distance layers

In [110]:
# create a profile for clipping

lc_profile.update({"dtype":'float32',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

In [112]:
vert_dist_simple = vert_dist_simple[0, :, :]
vert_dist_switchback = vert_dist_switchback[0, :, :]

In [113]:
# Clip the friction arrays by the buffered KP outline
# use a MemoryFile to avoid lots of IO -- otherwise have to save down, then load back up.

# dry
vert_dist_simple_mask, vert_dist_simple_mask_tform = rast_ops.clip_in_memory(vert_dist_simple,lc_profile,aoi.geometry)

# monsoon
vert_dist_switchback_mask, vert_dist_switchback_mask_tform = rast_ops.clip_in_memory(vert_dist_switchback,lc_profile,aoi.geometry)


In [114]:
exort_profile = lc_profile.copy()

export_profile.update({"dtype":'float32',\
                       "transform" : vert_dist_simple_mask_tform,\
                       "COMPRESS":'ZSTD',\
                       "nodata":-99999})

In [121]:
# write out the vertical-horizontal for future use

with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_Simple_{today}_{res}_masked.tif'),'w',**lc_profile) as dst1:
     dst1.write(vert_dist_simple_mask,indexes=1)
        
with rasterio.open(os.path.join(out_dir,f'KP_Combined_VertHorizontal_Distance_w_Switchbacks_{today}_{res}_masked.tif'),'w',**lc_profile) as dst2:
     dst2.write(vert_dist_switchback_mask,indexes=1)

## Merge rasters into final walking friction surface

Combine the various arrays into a final walking friction surface in 6 stages:
1. Multiply the base walking speed computed from the DEM Slope by the speed modifier
2. Create a monsoon walking speed as 0.75 of the base walking speed and the winter walking speed similarly, using a multiplier determined by elevation
3. Adjust the speeds for altitude
4. Transform these speeds into friction values
5. Multiply the friction values by the vert/horizontal multiplication factor (e.g. 1.5)
6. Convert extraneous values to -99999 nodata values

In [49]:
walkspeed_dry_step1 = np.divide(walkspeed_base,walkspeed_mod_rast)
walkspeed_msn_step1 = np.multiply(walkspeed_dry_step1,0.75)

In [50]:
walkspeed_dry_step1 = walkspeed_dry_step1.astype(np.float32)

In [51]:
export_profile = lc_profile.copy()
export_profile.update({"dtype":'float32',\
                       "COMPRESS":'ZSTD',
                       "NUM_THREADS":'ALL_CPUS',
                       "nodata":-99999})

In [52]:
# We impose differential winter speed costs based on altitude, on the expectation snowfall is a problem above 2350m and even more so above 3000m. 
# You can replace these with values that make sense for your context (or drop the winter model if that's not an issue)
walkspeed_winter_step0 = np.where(dem_array_reproj <= 2350, walkspeed_dry_step1, (np.multiply(walkspeed_dry_step1,0.75)))
walkspeed_winter_step1 = np.where(dem_array_reproj <= 3000, walkspeed_winter_step0, (np.multiply(walkspeed_dry_step1,0.6)))

In [53]:
np.min(walkspeed_winter_step1)

5.6437426e-07

Adjust walkspeeds by altitude

In [54]:
# We adjust altitude in two steps based on a literature review into how lower oxygen content at altitude affects walking speeds. Note this is not the best documented subject, at least in terms we can computer into a friction surface.
# This formula could probably be streamlined so that this step is condensed into one move
# The Global Friction Surface has just one formula but I found its high altitude (>5000) modifiers to be a little low compared to the available literature on athletic performance at altitude. Not a big deal except if you're working in the Himalayas

alt_adjust_dry_under3k = np.where(dem_array_reproj <= 2350, walkspeed_dry_step1, ((walkspeed_dry_step1) / (1 + ((dem_array_reproj - 2350)/5000))) )
walkspeed_dry_step2 = np.where(dem_array_reproj <= 3000, alt_adjust_dry_under3k, ((walkspeed_dry_step1) / (0.323 * np.exp((.00042*dem_array_reproj)))) )

alt_adjust_msn_under3k = np.where(dem_array_reproj <= 2350, walkspeed_msn_step1, ((walkspeed_msn_step1) / (1 + ((dem_array_reproj - 2350)/5000))) )
walkspeed_msn_step2 = np.where(dem_array_reproj <= 3000, alt_adjust_msn_under3k, ((walkspeed_msn_step1) / (0.323 * np.exp((.00042*dem_array_reproj)))) )

alt_adjust_winter_under3k = np.where(dem_array_reproj <= 2350, walkspeed_winter_step1, ((walkspeed_winter_step1) / (1 + ((dem_array_reproj - 2350)/5000))) )
walkspeed_winter_step2 = np.where(dem_array_reproj <= 3000, alt_adjust_winter_under3k, ((walkspeed_winter_step1) / (0.323 * np.exp((.00042*dem_array_reproj)))) )

In [55]:
# Refactor walking speeds to friction values in units of cell size / hour (e.g. 30m / hour)
# To prepare by minute instead just multiply by 60

friction_walk_dry_step1 = (1 / walkspeed_dry_step2) / (1000 / lc_transform.a)
friction_walk_msn_step1 = (1 / walkspeed_msn_step2) / (1000 / lc_transform.a)
friction_walk_winter_step1 = (1 / walkspeed_winter_step2) / (1000 / lc_transform.a)

In [66]:
# Now multiply the friction surface by the merged vertical/horizontal distance to calculate the final friction surface
friction_walk_dry_final = np.multiply(friction_walk_dry_step1,vert_dist_switchback)
friction_walk_msn_final = np.multiply(friction_walk_msn_step1,vert_dist_switchback)
friction_walk_winter_final = np.multiply(friction_walk_winter_step1,vert_dist_switchback)

In [67]:
#friction_walk_dry_final

In [68]:
#friction_walk_msn_final

In [69]:
#friction_walk_winter_final

Weed out Inf values and super high river values

In [70]:
# We use 1 as an arbitrary cutoff on the assumption that it will never actually take 1 hour to cross a grid cell, so values above that are bogus and filterable

friction_walk_dry_final = np.where(friction_walk_dry_final > 1, 1, friction_walk_dry_final)
friction_walk_msn_final = np.where(friction_walk_msn_final > 1, 1, friction_walk_msn_final)
friction_walk_winter_final = np.where(friction_walk_winter_final > 1, 1, friction_walk_winter_final)

Create a profile for clipping the layers

In [71]:
export_profile = lc_profile.copy()
export_profile.update({"dtype":'float32',\
                       "COMPRESS":'ZSTD',
                       "NUM_THREADS":'ALL_CPUS',
                       "nodata":-99999})

In [98]:
# remove band dimension for clipping operation as the function is built for indexless rasters
friction_walk_dry_final = friction_walk_dry_final[0, :, : ]
friction_walk_msn_final = friction_walk_msn_final[0, :, : ]
friction_walk_winter_final = friction_walk_winter_final[0, :, : ]

In [103]:
# Clip the friction arrays by the buffered KP outline
# use a MemoryFile to avoid lots of IO -- otherwise have to save down, then load back up.

# dry
friction_walk_dry_final_mask, friction_walk_dry_mask_tform = rast_ops.clip_in_memory(friction_walk_dry_final,export_profile,aoi.geometry)

# monsoon
friction_walk_msn_final_mask, friction_walk_msn_mask_tform = rast_ops.clip_in_memory(friction_walk_msn_final,export_profile,aoi.geometry)

# winter
friction_walk_winter_final_mask, friction_walk_winter_mask_tform = rast_ops.clip_in_memory(friction_walk_winter_final,export_profile,aoi.geometry)


Round up and change to float32 to reduce file sizes

In [105]:
# ensure it's float32 so the output size is smaller
friction_walk_dry_final_mask = np.round(friction_walk_dry_final_mask,8).astype(np.float32)
friction_walk_msn_final_mask = np.round(friction_walk_msn_final_mask,8).astype(np.float32)
friction_walk_winter_final_mask = np.round(friction_walk_winter_final_mask,8).astype(np.float32)

In [108]:
# make a new profile for export, inserting the new height/width/transform values from the clipped raster

mask_profile = export_profile.copy()

mask_profile.update({"dtype" : 'float32',
                     "height":friction_walk_winter_final_mask.shape[0],
                     "width":friction_walk_winter_final_mask.shape[1],
                     "transform" : friction_walk_winter_mask_tform})

In [ ]:
# Export the final, clipped version of the three seasonal friction surfaces

with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_dry_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_walk_dry_final_mask,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software
    
with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_msn_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_walk_msn_final_mask,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

with rasterio.open(os.path.join(fric_dir,f'KP_friction_walk_winter_{today}_{res}_masked.tif'),'w',**mask_profile) as dst:
    dst.write(friction_walk_winter_final_mask,indexes=1)
    dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software

# Troubleshooting: intermediate file export

It's easy to mistype a parameter and mess up the whole result. Looking through the intermediate files sequentially is usually the fastest way to catch and correct such errors. Below are quick code snippets to export these intermediate files.

In [ ]:
out_dir = r'P:\PAK\\Code\Accessibility\Intermediate_Rasters'

In [13]:
with rasterio.open(
        os.path.join(out_dir,f'KP_LC_Modifier.tif'), 'w',**lc_profile) as dst:
    dst.write(lc_array)

In [22]:
with rasterio.open(
        os.path.join(out_dir,f'KP_River_Raster.tif'), 'w',**lc_profile) as dst:
    dst.write(riv_rast, indexes=1)

In [51]:
slope = slope.astype('float64')
dem_profile.update({"dtype":'float64',\
                    "compress":'LZW'})

In [58]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'LZW',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,'KP_Slope.tif'),'w',**lc_profile) as dst:
     dst.write(slope)

In [ ]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'LZW',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,'KP_Walkspeed_base.tif'),'w',**lc_profile) as dst:
     dst.write(walkspeed_base)

In [109]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'LZW',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,'KP_Walkspeed_Modifier.tif'),'w',**lc_profile) as dst:
     dst.write(walkspeed_mod_rast)

In [54]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_alt_adjust_dry_under3k_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(alt_adjust_dry_under3k)

In [55]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_walkspeed_dry_step2_new_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(walkspeed_dry_step2)

In [56]:
#Temporary adjustment over 3k
temp_over3k = (0.323 * np.exp((.00042*dem_array_reproj)))

In [58]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_temp_over3k_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(temp_over3k)

In [57]:
temp_walkspeed_over3k = (walkspeed_dry_step1)/(0.323 * np.exp((.00042*dem_array_reproj)))

In [59]:
lc_profile.update({"dtype":'float64',\
                   "COMPRESS":'ZSTD',
                   "nodata":-99999})

with rasterio.open(os.path.join(out_dir,f'KP_temp_walkspeed_over3k_{today}.tif'),'w',**lc_profile) as dst:
     dst.write(temp_walkspeed_over3k)